# Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

'0.0.45'

In [2]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\MetaData'
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [3]:
import os.path
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,K21-59-FI#1,103.057281,1286.036078,590.183478,1286.036078,599.338630,0.435211,4.074877,2.728550e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,K21-59-FI#2,103.010651,1286.157501,227.288998,1286.157501,200.266823,0.409698,3.281104,2.195236e-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,K21-59-FI#3,103.046189,1286.074236,570.010451,1286.074236,580.897805,0.422397,3.585377,3.638637e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,K21-61-FI#1-rep1,103.310515,1285.431591,65.483825,1285.431591,81.303002,0.574737,3.109909,2.927106e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,K21-61-FI#1-rep2,103.304596,1285.435781,607.688675,1285.435781,895.513299,0.568316,3.571187,5.553914e-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'FG', 'Cap','window'],
 sort=True)
diad_meta

['K21-59-FI#1-IMG (1).txt',
 'K21-59-FI#1-IMG.txt',
 'K21-59-FI#1.txt',
 'K21-59-FI#2-IMG.txt',
 'K21-59-FI#2.txt',
 'K21-59-FI#3-IMG.txt',
 'K21-59-FI#3.txt',
 'K21-59-IMG5x.txt',
 'K21-60-IMG5x.txt',
 'K21-61-FI#1-IMG.txt',
 'K21-61-FI#1-rep1.txt',
 'K21-61-FI#1-rep2.txt',
 'K21-61-FI#2-IMG.txt',
 'K21-61-FI#2.txt',
 'K21-61-FI#3-IMG.txt',
 'K21-61-FI#3.txt',
 'K21-61-IMG5x.txt',
 'K21-62-FI#1-IMG.txt',
 'K21-62-FI#1-nodiad.txt',
 'K21-62-FI#2-IMG.txt',
 'K21-62-FI#2-nodiad.txt',
 'K21-62-IMG5x.txt',
 'K21-63-FI#1-IMG.txt',
 'K21-63-FI#1.txt',
 'K21-63-IMG5x.txt',
 'K21-64-FI#1-IMG.txt',
 'K21-64-FI#1.txt',
 'K21-64-FI#2-IMG.txt',
 'K21-64-FI#2.txt',
 'K21-64-FI#3-IMG.txt',
 'K21-64-FI#3.txt',
 'K21-64-FI#4-IMG.txt',
 'K21-64-FI#4.txt',
 'K21-64-FI#5-IMG.txt',
 'K21-64-FI#5.txt',
 'K21-64-FI#6-IMG.txt',
 'K21-64-FI#6.txt',
 'K21-64-FI#7-IMG.txt',
 'K21-64-FI#7.txt',
 'K21-64-FI#8-IMG.txt',
 'K21-64-FI#8.txt',
 'K21-64-IMG5x.txt']

## Now get all the important information out of the metadata files

In [7]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=False)
meta['filename'].iloc[0]

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 36.26it/s]

Done


'K21-59-FI#1.txt'

## Then get a simple file name you can stitch with the spectra

In [8]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

<class 'str'>
['K21-59-FI#1.txt' None None None None None None None None None None None
 None None None None None None]
<class 'str'>
['K21-59-FI#1' 'K21-59-FI#2.txt' None None None None None None None None
 None None None None None None None None]
<class 'str'>
['K21-59-FI#1' 'K21-59-FI#2' 'K21-59-FI#3.txt' None None None None None
 None None None None None None None None None None]
<class 'str'>
['K21-59-FI#1' 'K21-59-FI#2' 'K21-59-FI#3' 'K21-61-FI#1-rep1.txt' None
 None None None None None None None None None None None None None]
<class 'str'>
['K21-59-FI#1' 'K21-59-FI#2' 'K21-59-FI#3' 'K21-61-FI#1-rep1'
 'K21-61-FI#1-rep2.txt' None None None None None None None None None None
 None None None]
<class 'str'>
['K21-59-FI#1' 'K21-59-FI#2' 'K21-59-FI#3' 'K21-61-FI#1-rep1'
 'K21-61-FI#1-rep2' 'K21-61-FI#2.txt' None None None None None None None
 None None None None None]
<class 'str'>
['K21-59-FI#1' 'K21-59-FI#2' 'K21-59-FI#3' 'K21-61-FI#1-rep1'
 'K21-61-FI#1-rep2' 'K21-61-FI#2' 'K21-61-

'K21-59-FI#1'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

file_s

array(['K21-59-FI#1', 'K21-59-FI#2', 'K21-59-FI#3', 'K21-61-FI#1-rep1',
       'K21-61-FI#1-rep2', 'K21-63-FI#1', 'K21-64-FI#2', 'K21-64-FI#6',
       'K21-64-FI#8', 'K21-61-FI#2', 'K21-61-FI#3', 'K21-64-FI#1',
       'K21-64-FI#3', 'K21-64-FI#4', 'K21-64-FI#5', 'K21-64-FI#7'],
      dtype=object)

In [9]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [10]:
from pickle import load
import pickle
import os
pickle_path=os.getcwd()
file_pkl=pickle_path + '/' + 'Ne_Line_Regression_Example_cali.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [11]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

In [12]:
df_combo.to_excel('Calibration_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [13]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Peak_Cent_Carb,Peak_Area_Carb,Peak_Height_Carb
0,K21-59-FI#1.txt,1151.160659,57.033468,26.540273,NaN,NaN,NaN
1,K21-63-FI#1.txt,1150.591830,81.865421,56.150041,NaN,NaN,NaN
2,K21-64-FI#1.txt,1150.651543,690.347547,437.799964,NaN,NaN,NaN
3,K21-64-FI#2.txt,1150.642811,374.155731,235.883336,NaN,NaN,NaN
4,K21-64-FI#3.txt,1150.614040,1181.442087,738.919826,NaN,NaN,NaN


In [14]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

In [15]:
df_combo['name_for_matching']=df_combo['filename_x']
if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
else:
    df_combo_sec_phase=df_combo

In [16]:
df_combo_sec_phase.to_excel('Calibration_fitting_w_sec_phases.xlsx')